# 기계학습 (60점)

In [34]:
from IPython.display import display, HTML
display(HTML("<style>:root {--jp-notebook-max-width: 3000px;}</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

### 출생데이터
- 출처 : https://mdis.kostat.go.kr/dwnlSvc/ofrSurvSearch.do?curMenuNo=UI_POR_P9240
- 자료 이용 주의사항 (국가통계 자료제공 규정 제 17조 제 1항)     
  ○ 개별 자료에 의거 알게 된 사항에 대한 제공 및 누설 금지    
  ○ 자료 이용의 정확한 목적 명시 및 작성 목적 이외의 사용 금지     
  ○ 제공 자료의 활용이 끝난 후 즉시 파기    
  ○ 자료의 무단 공유 · 복제 금지     
  ○ 올바른 분석기법 사용 및 통계적 오차를 적정수준으로 유지하도록 노력     

- 본 모의고사는 MDIS 담당부서 문의 후 자료 이용 동의 받고 사용

데이터 경로 
- ./data/
    - 2019.csv ~ 2022.csv : 년도별 출생관련 데이터
    - index.csv : 데이터 컬럼 상세
        - 항목명 : 컬럼
            - 항목명 컬럼의 결측치는 값이 존재하는 상위 데이터와 일치한다. 
            - ex. 1행의 항목명은 '출생자주소지_행정구역시도코드' , 18행의 항목명은 성별코드
        - 코드 : 데이터
        - 코드의미 및 단위 : 코드의 의미

In [2]:
import pandas as pd
pd.set_option('display.max_columns',100)


df =pd.read_csv('./data/2019.csv')
display(df.head())
idx = pd.read_csv('./data/index.csv')
idx.head()

,연도,신고연도,신고월,신고일,출생자주소지_행정구역시도코드,성별코드,결혼중외의자녀여부코드,출생연도,출생월,출생장소코드,...,다태아분류코드,다태아출산순위코드,출생아체중,모총출생아수코드,모생존아수코드,부모동거기간,부_국적구분코드,부_국적코드,모_국적구분코드,모_국적코드
0,2019,2019,1,1,31,1,Y,2019,1,2,...,1,1,0.453,2,99,999,1,NaN,3,9000.0
1,2019,2019,1,1,33,1,9,2019,1,2,...,9,9,3.090,99,99,999,9,NaN,1,NaN
2,2019,2019,1,2,11,1,Y,2019,1,2,...,1,1,3.380,2,2,4,1,NaN,1,NaN
3,2019,2019,1,2,11,1,Y,2019,1,2,...,1,1,3.060,2,2,4,1,NaN,1,NaN
4,2019,2019,1,2,11,1,Y,2019,1,2,...,2,1,2.645,1,1,1,1,NaN,1,NaN


,항목명,코드,코드의미 및 단위
0,출생자주소지_행정구역시도코드,11,서울특별시
1,NaN,21,부산광역시
2,NaN,22,대구광역시
3,NaN,23,인천광역시
4,NaN,24,광주광역시


### 기계학습1
아래를 기준으로 데이터 전처리를 진행하라      
1. 년도별 출생 데이터를 하나의 데이터 프레임으로 합친다.     
2. 데이터 컬럼 상세 데이터를 바탕으로 출생 데이터의 값을 변환한다.
3. 변환한 데이터 컬럼 중 '미상'을 포함하는 행이 존재한다면 해당 행은 제거한다.    
4. 아래의 조건에 해당하는 각 컬럼의 값이 있다면 해당 행은 제거한다
    - 부_교육정도코드 : 학력없음    
    - 부_각세연령 : 999.0    
    - 모_교육정도코드 : 학력없음
    - 모_각세연령 :999.0
    - 실제결혼연도 : 9999.0
    - 실제결혼월 : 99.0
    - 부모동거기간 : 999
    - 모_각세연령 : 50세 이상
    - 임신주수 : 0
    - 출생아체중: 1kg 이하

결과예시
<img src="./data/1.png" style="width:10000px; float:left; ">

In [54]:
import numpy as np
import glob

# 출생데이터 통합
lst = []
for p in sorted(glob.glob('./data/*.csv'))[:-1]:
    lst.append(pd.read_csv(p))

t = pd.concat(lst).reset_index(drop=True)


# 상세 데이터 전천리
idx['항목명'] =idx['항목명'].fillna(method= 'ffill')

# 상세 데이터 딕셔너리
dics = {}

for part in idx['항목명'].unique():
    target = idx[idx['항목명'] == part]
    
    value_dics={}
    for k,v in target.iloc[:,1:].values:
        if k[0] =='0':
            value_dics[str(int(k))] = v
            
        else:
            value_dics[k] = v
        
    dics[part] = value_dics
    

# 출생데이터 메핑
tt = t.copy()
def convert_to_str(x):
    if pd.isna(x):  
        return np.nan
    
    elif isinstance(x,str):
        return x
    
    else:
        return str(int(x)) 


for col in t.columns:
    if col in dics.keys():
        
        tt[col] = tt[col].map(convert_to_str)
        tt[col] = tt[col].map(dics[col])

# 미상 데이터 제거
df= tt.copy()
for col in dics.keys():
    df = df[~df[col].astype('str').str.contains('미상')].reset_index(drop=True)

    
# 제시된 조건의 데이터 제거
remove_dic = {
    '부_교육정도코드' : '학력없음',
    '부_각세연령' : 999.0,
    '모_교육정도코드' : '학력없음',
    '모_각세연령' :999.0,
    '실제결혼연도' : 9999.0,
    '실제결혼월' : 99.0,
    '부모동거기간' : 999,
    '임신주수' : 0
}

for k,v in remove_dic.items():
    
    df = df[df[k] != v].reset_index(drop=True)
    
df= df[df['모_각세연령'] <50].reset_index(drop=True)
df= df[df['출생아체중'] >1].reset_index(drop=True)

In [55]:
df.head()

,연도,신고연도,신고월,신고일,출생자주소지_행정구역시도코드,성별코드,결혼중외의자녀여부코드,출생연도,출생월,출생장소코드,부_직업분류코드,부_교육정도코드,부_각세연령,모_직업분류코드,모_교육정도코드,모_각세연령,실제결혼연도,실제결혼월,임신주수,다태아분류코드,다태아출산순위코드,출생아체중,모총출생아수코드,모생존아수코드,부모동거기간,부_국적구분코드,부_국적코드,모_국적구분코드,모_국적코드
0,2019,2019,1,2,서울특별시,남자,혼인중의자,2019,1,병원,관리자,대학(교),33.0,전문가 및 관련 종사자,대학(교),32.0,2014.0,10.0,39,단태아,첫째,3.380,2명,2명,4,출생한국인,NaN,출생한국인,NaN
1,2019,2019,1,2,서울특별시,남자,혼인중의자,2019,1,병원,사무 종사자,대학(교),33.0,사무 종사자,대학(교),33.0,2014.0,4.0,39,단태아,첫째,3.060,2명,2명,4,출생한국인,NaN,출생한국인,NaN
2,2019,2019,1,2,서울특별시,남자,혼인중의자,2019,1,병원,서비스 종사자 및 판매 종사자,중학교,34.0,"학생, 가사, 무직",고등학교,34.0,2017.0,12.0,35,쌍태아,첫째,2.645,1명,1명,1,출생한국인,NaN,출생한국인,NaN
3,2019,2019,1,2,서울특별시,남자,혼인중의자,2019,1,병원,서비스 종사자 및 판매 종사자,중학교,34.0,"학생, 가사, 무직",고등학교,34.0,2017.0,12.0,35,쌍태아,둘째,2.335,2명,2명,1,출생한국인,NaN,출생한국인,NaN
4,2019,2019,1,2,서울특별시,남자,혼인중의자,2019,1,병원,서비스 종사자 및 판매 종사자,고등학교,34.0,단순노무 종사자,대학(교),33.0,2011.0,7.0,40,단태아,첫째,3.550,2명,2명,7,출생한국인,NaN,출생한국인,NaN


### 기계학습2
출생아체중을 중점으로 EDA를 실시하라 (시각화 포함)

### 기계학습3     
아래의 기준으로 추가적인 전처리를 실행하고 `출생아체중`이 각 컬럼들의 값에 따른 차이가 존재하는지 통계 검정을 실시하라
- 데이터 필터 : `신고월`이 2월이며 `신고일`이 10일 이하
- 컬럼제거 : 연도,신고연도,신고월,신고일, 출생자주소지_행정구역시도코드, 결혼중외의자녀여부코드,부_교육정도코드,모_직업분류코드,출생연도,출생월,출생장소코드,실제결혼연도,실제결혼월,모생존아수코드,부_국적코드,모_국적코드,모_국적구분코드,부_국적구분코드

In [87]:
strings = '연도,신고연도,신고월,신고일, 출생자주소지_행정구역시도코드, 결혼중외의자녀여부코드,부_교육정도코드,모_직업분류코드,출생연도,출생월,출생장소코드,실제결혼연도,실제결혼월,모생존아수코드,부_국적코드,모_국적코드,모_국적구분코드,부_국적구분코드'
remove_col = [x.strip() for x in strings.split(',')]
filter_col = [x for x in df.columns if x not in remove_col]


df_pre = df[(df.신고월 ==2) & (df.신고일 <=10)].reset_index(drop=True)
df_pre = df_pre[filter_col]

In [89]:
df_pre

,성별코드,부_직업분류코드,부_각세연령,모_교육정도코드,모_각세연령,임신주수,다태아분류코드,다태아출산순위코드,출생아체중,모총출생아수코드,부모동거기간
0,남자,관리자,31.000,대학원이상,31.000,40,단태아,첫째,3.180,1명,0
1,남자,관리자,33.000,대학(교),42.000,38,단태아,첫째,3.340,1명,1
2,남자,관리자,30.000,대학(교),30.000,38,단태아,첫째,3.930,1명,1
3,남자,관리자,33.000,대학(교),32.000,38,단태아,첫째,3.114,2명,4
4,남자,관리자,33.000,대학(교),33.000,38,단태아,첫째,3.790,1명,0
...,...,...,...,...,...,...,...,...,...,...,...
26190,여자,서비스 종사자 및 판매 종사자,24.416,고등학교,21.884,40,단태아,첫째,2.940,3명,0
26191,여자,전문가 및 관련 종사자,29.857,대학(교),28.361,38,단태아,첫째,3.480,2명,2
26192,여자,서비스 종사자 및 판매 종사자,35.917,고등학교,34.273,37,단태아,첫째,3.240,1명,0
26193,여자,"학생, 가사, 무직",29.523,대학(교),29.939,39,단태아,첫째,2.800,1명,7


### 기계학습4     
3에서 전처리한 데이터를 바탕으로 출생아 체중을 예측하는 회귀 모델을 만들려고한다. 아래의 조건에 맞게 모델링을 진행하고 평가하라
- 성별코드 기준으로 층화추출 진행 (학습데이터 70%, 평가 데이터 30%)
- 평가기준 rmse
- 랜덤포레스트, 다중선형회귀 모델로 진행

### 서울 지하철 일별 승하차 인원 데이터
출처 :https://data.seoul.go.kr/dataList/OA-12252/S/1/datasetView.do 후처리     
data_path = './data/subway.csv'


In [143]:
import pandas as pd
df = pd.read_csv('./data/subway.csv')
df.head()

,날짜,역번호,역명,구분,05~06,06~07,07~08,08~09,09~10,10~11,11~12,12~13,13~14,14~15,15~16,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~24,24~,호선
0,2016-01-01,150,서울역(150),승차,469,339,584,1144,1723,2087,2983,3527,3386,2612,3639,3554,3200,3076,2519,2281,2398,2056,996,58,1
1,2016-01-01,150,서울역(150),하차,342,1637,1753,1856,2438,2425,2548,2718,2973,2921,3341,3108,2656,2134,1859,1394,1344,1056,612,222,1
2,2016-01-01,151,시청(151),승차,103,127,110,191,298,582,700,604,731,900,1069,1339,1458,1499,1383,1284,1198,675,375,9,1
3,2016-01-01,151,시청(151),하차,68,306,554,939,689,665,884,956,1084,1216,1033,1035,951,792,618,402,309,192,114,34,1
4,2016-01-01,152,종각(152),승차,702,341,234,296,335,419,733,865,991,1300,1772,2175,2580,2627,2302,2315,2365,2140,1117,51,1


### 기계학습5    
아래의 기준으로 전처리를 진행하라     
1. 일자마다 노선별로 각 시간대의 모든역에서의 이용 승객 수를 승차,하차 각각 더한다.
    - 날짜,호선,구분,모든시간대 컬럼만 남아있어야한다. (4386 rows × 23 columns)
2. 아래의 조건으로 새로운 데이터프레임을 생성한다. 일자마다 호선별로 각 시간대의 승차 / 하차 비율을 구하여 _ratio 를 붙혀 새로운 컬럼을 만든다.
    - 2016-01-01, 1호선의 `05 ~ 06(승차) /05 ~ 06(하차)` : 
        - 컬럼 : `05~06_ratio` 
        - 값 : 3.5696 (2613/732)
3. 해당 일자가 평일(0)인지 주말(1)인지 구분하는 'weekend' 컬럼을 추가한다

<img src='./data/2.png'>

In [144]:
g = df.drop(columns =['역명','역번호']).groupby(['날짜','호선','구분']).sum().reset_index()

df_pivot = g.pivot_table(index=['날짜', '호선'], columns='구분', values=[x for x in g.columns if '~' in x])

lst = []
for date in [x for x in g.columns if '~' in x]:
    df_ratio = df_pivot[(date, '승차')] / df_pivot[(date, '하차')]
    df_ratio.name = f'{date}_ratio'
    lst.append(df_ratio)

dfs = pd.concat(lst,axis=1).reset_index()

dfs['날짜'] =pd.to_datetime(dfs['날짜'])
dfs['weekend']  = (dfs['날짜'].dt.weekday >=5).astype('int')
dfs.head()

,날짜,호선,05~06_ratio,06~07_ratio,07~08_ratio,08~09_ratio,09~10_ratio,10~11_ratio,11~12_ratio,12~13_ratio,13~14_ratio,14~15_ratio,15~16_ratio,16~17_ratio,17~18_ratio,18~19_ratio,19~20_ratio,20~21_ratio,21~22_ratio,22~23_ratio,23~24_ratio,24~_ratio,weekend
0,2016-01-01,1,3.569672,0.631725,0.548638,0.636792,0.775704,0.808009,0.831517,0.770004,0.766533,0.801024,1.045816,1.277316,1.353502,1.395663,1.458895,1.772087,1.944890,1.846209,1.311940,0.183721,0
1,2016-01-01,2,5.173876,1.198138,1.191171,1.100397,0.981767,1.037915,0.989607,0.968656,0.933618,0.944873,0.930663,0.979071,0.966366,0.934122,0.958119,1.079435,1.189924,1.170612,0.760431,0.004000,0
2,2016-01-01,3,3.814969,0.630485,0.782427,0.833200,0.759934,0.787604,0.828105,0.863113,0.862567,0.884181,1.018443,1.102005,1.154037,1.070333,1.050004,1.114768,1.227530,1.097164,0.506338,0.011957,0
3,2016-01-02,1,2.398010,0.591137,0.499784,0.582205,0.590733,0.642208,0.663121,0.686435,0.724733,0.818402,1.118137,1.348625,1.417669,1.380289,1.613574,2.016819,2.268172,2.443082,1.543539,0.177613,1
4,2016-01-02,2,3.903804,0.992869,1.161872,1.023701,0.870228,0.914519,0.916810,0.910801,0.898258,0.909322,0.949244,0.982930,0.995394,0.891454,0.949256,1.189477,1.190655,1.242373,0.776279,0.008628,1


### 기계학습6     
데이터 EDA를 진행하라. (시각화 포함)

### 기계학습7     
호선을 예측하는 분류 모델을 만들고 평가하라
- 평가데이터 : 일(day)이 4로 나눈 나머지가 1인 일자
- 학습데이터 : 그 외 
- 평가지표 : f1_score (micro,macro 모두 확인)
- 2가지 이상 모델

In [156]:
# test = dfs[(dfs['날짜'].dt.day % 4) ==1].reset_index(drop=True).drop(columns ='날짜')
# train = dfs[(dfs['날짜'].dt.day % 4) !=1].reset_index(drop=True).drop(columns ='날짜')


test = dfs[dfs['날짜'].dt.year ==2017].reset_index(drop=True).drop(columns ='날짜')
train = dfs[dfs['날짜'].dt.year ==2016].reset_index(drop=True).drop(columns ='날짜')



train_x = train.drop(columns ='호선')
train_y = train['호선']


test_x = test.drop(columns ='호선')
test_y = test['호선']

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(random_state=1)
dt = DecisionTreeClassifier(random_state=1)

rf.fit(train_x,train_y)
rf_predict = rf.predict(test_x)

dt.fit(train_x,train_y)
dt_predict = dt.predict(test_x)

rf_f1_mic = f1_score(test_y,rf_predict,average='micro')
dt_f1_mic = f1_score(test_y,dt_predict,average='micro')

rf_f1_mac = f1_score(test_y,rf_predict,average='macro')
dt_f1_mac = f1_score(test_y,dt_predict,average='macro')

result = pd.DataFrame()

result['model'] = ['RandomForest','DecisionTree']
result['micro'] = [rf_f1_mic,dt_f1_mic]
result['macro'] = [rf_f1_mac,dt_f1_mac]
result.set_index('model',inplace=True)
result

,micro,macro
model,,
RandomForest,0.999087,0.999087
DecisionTree,0.986301,0.986296


# 통계 (40점)

### 통계1      
한 카페에서는 고객들이 음료를 주문한 후, 음료가 준비되는 시간을 추적한다.      
음료 준비 시간은 평균이 5분이고, 표준편차가 1.5분인 정규분포를 따른다.         

1) 특정 고객이 음료를 6분 이내에 받을 확률은?     
2) 3분에서 7분 사이에 음료가 준비될 확률은?   
3) 10명의 고객 중, 6분 이내에 음료를 받을 고객이 3명 이상일 확률은?

In [159]:
import numpy as np
import scipy.stats as stats

# 1. 음료 준비 시간이 평균 5분, 표준편차 1.5분인 정규분포에서 6분 이내에 받을 확률
mu = 5 
sigma = 1.5  

# 6분 이내에 받을 확률
prob_6min = stats.norm.cdf(6, mu, sigma)
print(f"1. 6분 이내에 음료를 받을 확률: {prob_6min:.4f}")

# 2. 3분에서 7분 사이에 음료가 준비될 확률
prob_3_to_7 = stats.norm.cdf(7, mu, sigma) - stats.norm.cdf(3, mu, sigma)
print(f"2. 3분에서 7분 사이에 음료를 받을 확률: {prob_3_to_7:.4f}")

# 3. 10명의 고객 중, 6분 이내에 음료를 받을 고객이 3명 이상일 확률 
n = 10  
p = prob_6min  

# 이항분포에서 3명 이상일 확률 계산
prob_3_or_more = 1 - stats.binom.cdf(2, n, p)
print(f"3. 10명의 고객 중 3명 이상이 6분 이내에 음료를 받을 확률: {prob_3_or_more:.4f}")


1. 6분 이내에 음료를 받을 확률: 0.7475
2. 3분에서 7분 사이에 음료를 받을 확률: 0.8176
3. 10명의 고객 중 3명 이상이 6분 이내에 음료를 받을 확률: 0.9996


### 통계2
한 레스토랑에서 두 명의 셰프(A와 B)가 준비한 요리에 대한 고객들의 만족도를 조사했다.     
각 셰프가 준비한 요리에 대해 10명의 고객이 점수를 매겼다. 점수는 1점부터 10점까지이며, 점수가 높을수록 만족도가 높음을 의미한다.     
두 셰프의 요리에 대한 고객 점수가 정규분포를 따르지 않는다고 가정하고, 셰프 A와 셰프 B의 요리에 대해 고객들의 만족도가 차이가 나는지 비모수 검정을 통해 확인하라.     

셰프 A의 고객 점수: 
7,8,9,5,6,7,8,6,7,9

셰프 B의 고객 점수: 
6,5,8,7,6,5,6,4,7,5

(1) 가설설정     
(2) 통계검정

In [161]:
import numpy as np
from scipy.stats import mannwhitneyu

# 셰프 A와 셰프 B의 고객 점수
scores_A = np.array([7, 8, 9, 5, 6, 7, 8, 6, 7, 9])
scores_B = np.array([6, 5, 8, 7, 6, 5, 6, 4, 7, 5])

# Mann-Whitney U 검정 수행
stat, p_value = mannwhitneyu(scores_A, scores_B, alternative='two-sided')

print(f"Mann-Whitney U 통계량: {stat}")
print(f"p-value: {p_value:.4f}")

# p-value에 따른 결론 도출
alpha = 0.05  # 유의수준 5%
if p_value < alpha:
    print("두 셰프의 요리에 대한 만족도에 유의미한 차이가 있다.")
else:
    print("두 셰프의 요리에 대한 만족도에 유의미한 차이가 없다.")


Mann-Whitney U 통계량: 76.5
p-value: 0.0447
두 셰프의 요리에 대한 만족도에 유의미한 차이가 있다.


### 통계3       
병원에서 특정 질병의 진단을 위해 두 가지 서로 다른 검사 (검사 X, 검사 Y)를 시행한다. 검사 X와 Y는 서로 독립적이며 각각 다른 진단 결과를 제공한다.    


- 전체 인구에서 질병을 가진 사람의 비율: 2% 
- 질병이 없는 사람의 비율: 98% 
- 질병이 있는 경우 검사 X가 양성일 확률: 73% 
- 질병이 없는 경우 검사 X가 양성일 확률: 10% 
- 질병이 있는 경우 검사 Y가 양성일 확률: 66% 
- 질병이 없는 경우 검사 Y가 양성일 확률: 5% 


1. 검사 X와 검사 Y 모두 양성일 때, 환자가 실제로 질병을 가질 확률은 얼마인가?
2. 검사 X가 양성이고 검사 Y가 음성일 때, 환자가 실제로 질병을 가질 확률은 얼마인가?
3. 검사 Y가 양성일 때, 검사 X가 음성일 확률은?

In [162]:
from scipy.stats import binom
import numpy as np

# 주어진 확률들
P_D = 0.02        # 질병이 있을 확률
P_not_D = 0.98    # 질병이 없을 확률

P_X_pos_D = 0.73  # 질병이 있을 때 검사 X가 양성일 확률
P_X_pos_not_D = 0.10  # 질병이 없을 때 검사 X가 양성일 확률

P_Y_pos_D = 0.66  # 질병이 있을 때 검사 Y가 양성일 확률
P_Y_pos_not_D = 0.05  # 질병이 없을 때 검사 Y가 양성일 확률

# 1. P(D | X+ ∩ Y+): 검사 X와 Y 모두 양성일 때, 질병이 있을 확률

# P(X+ ∩ Y+ | D) = P(X+ | D) * P(Y+ | D) (독립 가정)
P_XY_pos_D = P_X_pos_D * P_Y_pos_D

# P(X+ ∩ Y+ | ¬D) = P(X+ | ¬D) * P(Y+ | ¬D)
P_XY_pos_not_D = P_X_pos_not_D * P_Y_pos_not_D

# P(X+ ∩ Y+) = P(X+ ∩ Y+ | D) * P(D) + P(X+ ∩ Y+ | ¬D) * P(¬D)
P_XY_pos = P_XY_pos_D * P_D + P_XY_pos_not_D * P_not_D

# P(D | X+ ∩ Y+) = (P(X+ ∩ Y+ | D) * P(D)) / P(X+ ∩ Y+)
P_D_given_XY_pos = (P_XY_pos_D * P_D) / P_XY_pos


# 2. P(D | X+ ∩ Y-): 검사 X가 양성이고 Y가 음성일 때, 질병이 있을 확률

# P(X+ ∩ Y- | D) = P(X+ | D) * P(Y- | D) = P(X+ | D) * (1 - P(Y+ | D))
P_X_pos_Y_neg_D = P_X_pos_D * (1 - P_Y_pos_D)

# P(X+ ∩ Y- | ¬D) = P(X+ | ¬D) * P(Y- | ¬D) = P(X+ | ¬D) * (1 - P(Y+ | ¬D))
P_X_pos_Y_neg_not_D = P_X_pos_not_D * (1 - P_Y_pos_not_D)

# P(X+ ∩ Y-) = P(X+ ∩ Y- | D) * P(D) + P(X+ ∩ Y- | ¬D) * P(¬D)
P_X_pos_Y_neg = P_X_pos_Y_neg_D * P_D + P_X_pos_Y_neg_not_D * P_not_D

# P(D | X+ ∩ Y-) = (P(X+ ∩ Y- | D) * P(D)) / P(X+ ∩ Y-)
P_D_given_X_pos_Y_neg = (P_X_pos_Y_neg_D * P_D) / P_X_pos_Y_neg


# 3. P(X- | Y+): 검사 Y가 양성일 때 검사 X가 음성일 확률

# P(X- ∩ Y+ | D) = P(X- | D) * P(Y+ | D) = (1 - P(X+ | D)) * P(Y+ | D)
P_X_neg_Y_pos_D = (1 - P_X_pos_D) * P_Y_pos_D

# P(X- ∩ Y+ | ¬D) = P(X- | ¬D) * P(Y+ | ¬D) = (1 - P(X+ | ¬D)) * P(Y+ | ¬D)
P_X_neg_Y_pos_not_D = (1 - P_X_pos_not_D) * P_Y_pos_not_D

# P(X- ∩ Y+) = P(X- ∩ Y+ | D) * P(D) + P(X- ∩ Y+ | ¬D) * P(¬D)
P_X_neg_Y_pos = P_X_neg_Y_pos_D * P_D + P_X_neg_Y_pos_not_D * P_not_D

# P(Y+) = P(Y+ | D) * P(D) + P(Y+ | ¬D) * P(¬D)
P_Y_pos = P_Y_pos_D * P_D + P_Y_pos_not_D * P_not_D

# P(X- | Y+) = P(X- ∩ Y+) / P(Y+)
P_X_neg_given_Y_pos = P_X_neg_Y_pos / P_Y_pos

P_D_given_XY_pos, P_D_given_X_pos_Y_neg, P_X_neg_given_Y_pos


(0.662905888827738, 0.05062000326317507, 0.7663022508038585)